In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qa-csv/qa.csv


In [110]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from numpy.random import seed

import pandas as pd
import numpy as np
import string, os 

In [3]:
!pip install spacy_hunspell

  Created wheel for spacy-hunspell: filename=spacy_hunspell-0.1.0-py3-none-any.whl size=3055 sha256=2e6d514a114f95ef24960a4ae41e9092eb79373bac00bfd853144e51a73b8f63
  Stored in directory: /root/.cache/pip/wheels/13/df/f1/c2e3798fd69bc7f8e76bdfd47bdf3727dbc8db958718bd7269
  Created wheel for hunspell: filename=hunspell-0.5.0-cp37-cp37m-linux_x86_64.whl size=60803 sha256=0a5fe16694da7a453825710538ce35abcbf0141c8c1bd8bda6db1a4f654fd644
  Stored in directory: /root/.cache/pip/wheels/11/9b/6f/a2c256f09391480ade0bf793ec749a0989e489717f5b45911f
Successfully built spacy-hunspell hunspell
  Attempting uninstall: hunspell
    Found existing installation: hunspell 0.5.5
    Uninstalling hunspell-0.5.5:
      Successfully uninstalled hunspell-0.5.5


In [4]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 6.9 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [6]:
import spacy
from spacy_hunspell import spaCyHunSpell

In [7]:
nlp = spacy.load('en_core_web_sm')
hunspell = spaCyHunSpell(nlp, ('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff'))

In [8]:
df = pd.read_csv('../input/qa-csv/qa.csv', delimiter='\t', encoding='utf-8', index_col=0)

In [9]:
df.columns

Index(['Query'], dtype='object')

In [10]:
df['Query'] = df['Query'].fillna('')

In [11]:
df

,Query
0,bag
1,Fendi handbag
2,Fendi beige handbag
3,backpacks
4,black backpacks
...,...
608,boys hoodies
609,onesie
610,baby shoes
611,baby top and shorts set


In [12]:
queries = list(df['Query'])

In [13]:
queries

['bag',
 'Fendi handbag',
 'Fendi beige handbag',
 'backpacks',
 'black backpacks',
 'blue bucket bag',
 'bucketbag',
 'crocodile print bag',
 'animal print bag',
 'tote bag',
 'totes',
 'FURLA pochette',
 'red CELINE crossbody ',
 'Coach work bag',
 'pink coccinelle bum bag',
 'belt bag',
 'Burberry duffel bag',
 'faux leather wallet',
 'pale pink wallet',
 'Floral print midi dress',
 'red sheath dress',
 'little black dress',
 'yellow short sleeve mini dress',
 'burgundy crewneck dress',
 'crew neck dress',
 'Off Shoulder Dress',
 'A line dress',
 'animal print dress',
 'leopard print dress',
 'lace dress',
 'shirt dress',
 ' striped knee length dress',
 'full sleeve dress',
 'shift sack dress',
 'maison margiela long dress',
 'polka dot mini dress',
 'needle & thread black dress',
 'flared dress',
 ' DRESS',
 'red off shoulder midi dress',
 'alberta ferreti dress',
 'black maxi dress',
 'Bodycon dress',
 'Alice Mccall midi dress',
 'strapless dress',
 'yellow dress',
 'reem acra max

In [14]:
queries_updated = []
for query in queries:
    query = query.strip()
    split_len = len(query.split(' '))
    if split_len >= 2:
        queries_updated.append(query)

In [15]:
len(queries_updated)

541

In [16]:
# df = df.drop(['Query'], axis=1)
del df

In [17]:
df = pd.DataFrame(queries_updated, columns=['Query'])

In [18]:
# df.reset_index()

In [19]:
df.columns

Index(['Query'], dtype='object')

In [20]:
df.columns

Index(['Query'], dtype='object')

In [21]:
len(df)

541

In [22]:
tokenizer = Tokenizer()
# def get_sequence_of_tokens(corpus):
#     pass
tokenizer.fit_on_texts(queries_updated)

In [23]:
total_words = len(tokenizer.word_index) + 1
total_words

481

In [24]:
input_sequences = []
for line in queries_updated:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [25]:
input_sequences

[[71, 169],
 [71, 123],
 [71, 123, 169],
 [2, 170],
 [18, 95],
 [18, 95, 14],
 [285, 4],
 [285, 4, 14],
 [34, 4],
 [34, 4, 14],
 [96, 14],
 [286, 287],
 [8, 288],
 [8, 288, 289],
 [290, 291],
 [290, 291, 14],
 [72, 292],
 [72, 292, 293],
 [72, 292, 293, 14],
 [29, 14],
 [294, 295],
 [294, 295, 14],
 [97, 3],
 [97, 3, 73],
 [296, 72],
 [296, 72, 73],
 [52, 4],
 [52, 4, 23],
 [52, 4, 23, 1],
 [8, 171],
 [8, 171, 1],
 [172, 2],
 [172, 2, 1],
 [35, 53],
 [35, 53, 19],
 [35, 53, 19, 24],
 [35, 53, 19, 24, 1],
 [173, 297],
 [173, 297, 1],
 [298, 54],
 [298, 54, 1],
 [55, 16],
 [55, 16, 1],
 [174, 175],
 [174, 175, 1],
 [34, 4],
 [34, 4, 1],
 [98, 4],
 [98, 4, 1],
 [99, 1],
 [11, 1],
 [74, 56],
 [74, 56, 41],
 [74, 56, 41, 1],
 [25, 19],
 [25, 19, 1],
 [299, 300],
 [299, 300, 1],
 [124, 125],
 [124, 125, 176],
 [124, 125, 176, 1],
 [75, 76],
 [75, 76, 24],
 [75, 76, 24, 1],
 [177, 178],
 [177, 178, 2],
 [177, 178, 2, 1],
 [126, 1],
 [8, 55],
 [8, 55, 16],
 [8, 55, 16, 23],
 [8, 55, 16, 23, 1]

In [26]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
for i, j in zip(predictors, label):
    print(i, j)
label = ku.to_categorical(label, num_classes=total_words)

[ 0  0  0  0  0 71] 169
[ 0  0  0  0  0 71] 123
[  0   0   0   0  71 123] 169
[0 0 0 0 0 2] 170
[ 0  0  0  0  0 18] 95
[ 0  0  0  0 18 95] 14
[  0   0   0   0   0 285] 4
[  0   0   0   0 285   4] 14
[ 0  0  0  0  0 34] 4
[ 0  0  0  0 34  4] 14
[ 0  0  0  0  0 96] 14
[  0   0   0   0   0 286] 287
[0 0 0 0 0 8] 288
[  0   0   0   0   8 288] 289
[  0   0   0   0   0 290] 291
[  0   0   0   0 290 291] 14
[ 0  0  0  0  0 72] 292
[  0   0   0   0  72 292] 293
[  0   0   0  72 292 293] 14
[ 0  0  0  0  0 29] 14
[  0   0   0   0   0 294] 295
[  0   0   0   0 294 295] 14
[ 0  0  0  0  0 97] 3
[ 0  0  0  0 97  3] 73
[  0   0   0   0   0 296] 72
[  0   0   0   0 296  72] 73
[ 0  0  0  0  0 52] 4
[ 0  0  0  0 52  4] 23
[ 0  0  0 52  4 23] 1
[0 0 0 0 0 8] 171
[  0   0   0   0   8 171] 1
[  0   0   0   0   0 172] 2
[  0   0   0   0 172   2] 1
[ 0  0  0  0  0 35] 53
[ 0  0  0  0 35 53] 19
[ 0  0  0 35 53 19] 24
[ 0  0 35 53 19 24] 1
[  0   0   0   0   0 173] 297
[  0   0   0   0 173 297] 1
[  0   0  

[  0   0   0   0 218 219] 401
[  0   0   0 218 219 401] 10
[ 0  0  0  0  0 59] 2
[ 0  0  0  0 59  2] 42
[ 0  0  0 59  2 42] 10
[  0   0   0   0   0 402] 272
[  0   0   0   0 402 272] 10
[  0   0   0   0   0 403] 404
[  0   0   0   0 403 404] 207
[  0   0   0 403 404 207] 10
[  0   0   0   0   0 405] 6
[  0   0   0   0   0 406] 6
[ 0  0  0  0  0 77] 20
[ 0  0  0  0 77 20] 6
[  0   0   0   0   0 162] 6
[ 0  0  0  0  0 56] 41
[ 0  0  0  0 56 41] 6
[  0   0   0   0   0 208] 6
[0 0 0 0 0 5] 6
[ 0  0  0  0  0 53] 6
[ 0  0  0  0  0 59] 2
[ 0  0  0  0 59  2] 6
[  0   0   0   0   0 209] 210
[  0   0   0   0   0 211] 3
[  0   0   0   0 211   3] 17
[0 0 0 0 0 3] 212
[  0   0   0   0   3 212] 17
[  0   0   0   0   0 110] 276
[  0   0   0   0 110 276] 213
[  0   0   0 110 276 213] 17
[  0   0   0   0   0 214] 17
[0 0 0 0 0 8] 60
[0 0 0 0 0 2] 60
[  0   0   0   0   0 407] 408
[  0   0   0   0 407 408] 60
[ 0  0  0  0  0 84] 215
[  0   0   0   0   0 138] 17
[  0   0   0   0   0 216] 111
[ 0  0  0  0 

In [27]:
queries_updated

['Fendi handbag',
 'Fendi beige handbag',
 'black backpacks',
 'blue bucket bag',
 'crocodile print bag',
 'animal print bag',
 'tote bag',
 'FURLA pochette',
 'red CELINE crossbody',
 'Coach work bag',
 'pink coccinelle bum bag',
 'belt bag',
 'Burberry duffel bag',
 'faux leather wallet',
 'pale pink wallet',
 'Floral print midi dress',
 'red sheath dress',
 'little black dress',
 'yellow short sleeve mini dress',
 'burgundy crewneck dress',
 'crew neck dress',
 'Off Shoulder Dress',
 'A line dress',
 'animal print dress',
 'leopard print dress',
 'lace dress',
 'shirt dress',
 'striped knee length dress',
 'full sleeve dress',
 'shift sack dress',
 'maison margiela long dress',
 'polka dot mini dress',
 'needle & thread black dress',
 'flared dress',
 'red off shoulder midi dress',
 'alberta ferreti dress',
 'black maxi dress',
 'Bodycon dress',
 'Alice Mccall midi dress',
 'strapless dress',
 'yellow dress',
 'reem acra maxi dress',
 'Halterneck Dress',
 'a line v neck dress',
 'al

In [28]:
queries_updated_tags = []

In [29]:
for query in queries_updated:
    nlp_text = nlp(query)
    token_list = []
    for token in nlp_text:
        token_list.append(token.pos_)
    nlp_pos = " ".join(token_list)
    queries_updated_tags.append(nlp_pos)

In [30]:
queries_updated_tags

['PROPN NOUN',
 'PROPN ADJ NOUN',
 'ADJ NOUN',
 'ADJ NOUN NOUN',
 'NOUN NOUN NOUN',
 'NOUN NOUN NOUN',
 'PROPN NOUN',
 'PROPN PROPN',
 'PROPN PROPN NOUN',
 'NOUN NOUN NOUN',
 'ADJ PROPN PROPN PROPN',
 'PROPN PROPN',
 'PROPN NOUN NOUN',
 'PROPN NOUN NOUN',
 'ADJ ADJ NOUN',
 'ADJ NOUN NOUN NOUN',
 'ADJ NOUN NOUN',
 'ADJ ADJ NOUN',
 'PROPN ADJ NOUN ADJ NOUN',
 'PROPN PROPN NOUN',
 'NOUN NOUN NOUN',
 'PROPN PROPN PROPN',
 'DET NOUN NOUN',
 'NOUN NOUN NOUN',
 'NOUN NOUN NOUN',
 'PROPN PROPN',
 'NOUN PROPN',
 'VERB NOUN NOUN NOUN',
 'ADJ NOUN NOUN',
 'VERB NOUN NOUN',
 'PROPN PROPN ADV NOUN',
 'ADJ NOUN NOUN NOUN',
 'PROPN CCONJ PROPN ADJ NOUN',
 'VERB NOUN',
 'PROPN ADP PROPN PROPN NOUN',
 'PROPN PROPN NOUN',
 'ADJ PROPN NOUN',
 'NOUN VERB',
 'PROPN PROPN NOUN NOUN',
 'NOUN NOUN',
 'ADJ NOUN',
 'PROPN PROPN PROPN NOUN',
 'PROPN NOUN',
 'DET NOUN ADP NOUN NOUN',
 'PROPN PROPN PROPN NOUN',
 'PROPN NOUN',
 'ADJ ADJ NOUN',
 'PROPN PROPN ADJ NOUN',
 'PROPN PROPN PROPN',
 'NUM NOUN NOUN',
 'ADJ P

In [31]:
tokenizer_pos = Tokenizer()
tokenizer_pos.fit_on_texts(queries_updated_tags)

In [32]:
total_words_pos = len(tokenizer_pos.word_index) + 1
total_words_pos

12

In [35]:
input_sequences_pos = []
for line in queries_updated_tags:
    token_list = tokenizer_pos.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences_pos.append(n_gram_sequence)

In [36]:
input_sequences_pos

[[2, 1],
 [2, 3],
 [2, 3, 1],
 [3, 1],
 [3, 1],
 [3, 1, 1],
 [1, 1],
 [1, 1, 1],
 [1, 1],
 [1, 1, 1],
 [2, 1],
 [2, 2],
 [2, 2],
 [2, 2, 1],
 [1, 1],
 [1, 1, 1],
 [3, 2],
 [3, 2, 2],
 [3, 2, 2, 2],
 [2, 2],
 [2, 1],
 [2, 1, 1],
 [2, 1],
 [2, 1, 1],
 [3, 3],
 [3, 3, 1],
 [3, 1],
 [3, 1, 1],
 [3, 1, 1, 1],
 [3, 1],
 [3, 1, 1],
 [3, 3],
 [3, 3, 1],
 [2, 3],
 [2, 3, 1],
 [2, 3, 1, 3],
 [2, 3, 1, 3, 1],
 [2, 2],
 [2, 2, 1],
 [1, 1],
 [1, 1, 1],
 [2, 2],
 [2, 2, 2],
 [10, 1],
 [10, 1, 1],
 [1, 1],
 [1, 1, 1],
 [1, 1],
 [1, 1, 1],
 [2, 2],
 [1, 2],
 [4, 1],
 [4, 1, 1],
 [4, 1, 1, 1],
 [3, 1],
 [3, 1, 1],
 [4, 1],
 [4, 1, 1],
 [2, 2],
 [2, 2, 11],
 [2, 2, 11, 1],
 [3, 1],
 [3, 1, 1],
 [3, 1, 1, 1],
 [2, 9],
 [2, 9, 2],
 [2, 9, 2, 3],
 [2, 9, 2, 3, 1],
 [4, 1],
 [2, 8],
 [2, 8, 2],
 [2, 8, 2, 2],
 [2, 8, 2, 2, 1],
 [2, 2],
 [2, 2, 1],
 [3, 2],
 [3, 2, 1],
 [1, 4],
 [2, 2],
 [2, 2, 1],
 [2, 2, 1, 1],
 [1, 1],
 [3, 1],
 [2, 2],
 [2, 2, 2],
 [2, 2, 2, 1],
 [2, 1],
 [10, 1],
 [10, 1, 8],
 [10, 1, 8

In [37]:
input_sequences_pos

[[2, 1],
 [2, 3],
 [2, 3, 1],
 [3, 1],
 [3, 1],
 [3, 1, 1],
 [1, 1],
 [1, 1, 1],
 [1, 1],
 [1, 1, 1],
 [2, 1],
 [2, 2],
 [2, 2],
 [2, 2, 1],
 [1, 1],
 [1, 1, 1],
 [3, 2],
 [3, 2, 2],
 [3, 2, 2, 2],
 [2, 2],
 [2, 1],
 [2, 1, 1],
 [2, 1],
 [2, 1, 1],
 [3, 3],
 [3, 3, 1],
 [3, 1],
 [3, 1, 1],
 [3, 1, 1, 1],
 [3, 1],
 [3, 1, 1],
 [3, 3],
 [3, 3, 1],
 [2, 3],
 [2, 3, 1],
 [2, 3, 1, 3],
 [2, 3, 1, 3, 1],
 [2, 2],
 [2, 2, 1],
 [1, 1],
 [1, 1, 1],
 [2, 2],
 [2, 2, 2],
 [10, 1],
 [10, 1, 1],
 [1, 1],
 [1, 1, 1],
 [1, 1],
 [1, 1, 1],
 [2, 2],
 [1, 2],
 [4, 1],
 [4, 1, 1],
 [4, 1, 1, 1],
 [3, 1],
 [3, 1, 1],
 [4, 1],
 [4, 1, 1],
 [2, 2],
 [2, 2, 11],
 [2, 2, 11, 1],
 [3, 1],
 [3, 1, 1],
 [3, 1, 1, 1],
 [2, 9],
 [2, 9, 2],
 [2, 9, 2, 3],
 [2, 9, 2, 3, 1],
 [4, 1],
 [2, 8],
 [2, 8, 2],
 [2, 8, 2, 2],
 [2, 8, 2, 2, 1],
 [2, 2],
 [2, 2, 1],
 [3, 2],
 [3, 2, 1],
 [1, 4],
 [2, 2],
 [2, 2, 1],
 [2, 2, 1, 1],
 [1, 1],
 [3, 1],
 [2, 2],
 [2, 2, 2],
 [2, 2, 2, 1],
 [2, 1],
 [10, 1],
 [10, 1, 8],
 [10, 1, 8

In [45]:
max_sequence_len = max([len(x) for x in input_sequences_pos])
input_sequences_pos = np.array(pad_sequences(input_sequences_pos, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences_pos[:,:-1],input_sequences_pos[:,-1]
for i, j in zip(predictors, label):
    print(i, j)
label = ku.to_categorical(label, num_classes=total_words_pos)

[0 0 0 0 0 0 2] 1
[0 0 0 0 0 0 2] 3
[0 0 0 0 0 2 3] 1
[0 0 0 0 0 0 3] 1
[0 0 0 0 0 0 3] 1
[0 0 0 0 0 3 1] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 3] 2
[0 0 0 0 0 3 2] 2
[0 0 0 0 3 2 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 2 1] 1
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 2 1] 1
[0 0 0 0 0 0 3] 3
[0 0 0 0 0 3 3] 1
[0 0 0 0 0 0 3] 1
[0 0 0 0 0 3 1] 1
[0 0 0 0 3 1 1] 1
[0 0 0 0 0 0 3] 1
[0 0 0 0 0 3 1] 1
[0 0 0 0 0 0 3] 3
[0 0 0 0 0 3 3] 1
[0 0 0 0 0 0 2] 3
[0 0 0 0 0 2 3] 1
[0 0 0 0 2 3 1] 3
[0 0 0 2 3 1 3] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 2
[ 0  0  0  0  0  0 10] 1
[ 0  0  0  0  0 10  1] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 1] 2
[0 0 0 0 0 0 4] 1
[0 0 0 0 0 4 1] 1
[0 0 0 0 4 1 1] 1
[0 0 0 0 0 0 3

[0 0 0 0 0 2 2] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 1
[0 0 0 0 2 2 1] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 0 2] 4
[0 0 0 0 0 0 3] 1
[0 0 0 0 0 3 1] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 2 1] 1
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 0 3] 1
[0 0 0 0 0 0 3] 3
[0 0 0 0 0 3 3] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 8
[0 0 0 0 2 2 8] 2
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 5
[0 0 0 0 0 2 5] 2
[0 0 0 0 0 0 3] 3
[0 0 0 0 0 3 3] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 1
[0 0 0 0 0 0 4] 1
[0 0 0 0 0 0 2] 4
[0 0 0 0 0 2 4] 1
[0 0 0 0 2 4 1] 1
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 0 3] 1
[0 0 0 0 0 3 1] 1
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 2
[0 0 0 0 0 0 2] 2
[0 0 0 0 0 2 2] 2
[0 0 0 0 2 2 2] 2
[0 0 0 0 0 0 4] 2
[0 0 0 0 0 0 1] 1
[0 0 0 0 0 1 1] 1
[0 0 0 0 0 0 3] 5
[0 0 0 0 0 3 5] 1
[0 0 0 0 3

In [46]:
len(label)

1089

In [47]:
print(label[1])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [53]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
    
    return model

In [66]:
model = create_model(max_sequence_len, total_words_pos)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 7, 10)             120       
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 12)                1212      
Total params: 45,732
Trainable params: 45,732
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.fit(predictors, label, epochs=5)

Epoch 1/5
35/35 [==============================] - 2s 5ms/step - loss: 2.2714 - accuracy: 0.4720
Epoch 2/5
35/35 [==============================] - 0s 5ms/step - loss: 1.2234 - accuracy: 0.5090
Epoch 3/5
35/35 [==============================] - 0s 5ms/step - loss: 1.2505 - accuracy: 0.5259
Epoch 4/5
35/35 [==============================] - 0s 4ms/step - loss: 1.2920 - accuracy: 0.5307
Epoch 5/5
35/35 [==============================] - 0s 4ms/step - loss: 1.2670 - accuracy: 0.5258


In [100]:
def convert_into_pos_tag(query):
    nlp_text = nlp(query)
    token_list = []
    for token in nlp_text:
        token_list.append(token.pos_)
    nlp_pos = " ".join(token_list)
    return nlp_pos

In [155]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    seed_text = convert_into_pos_tag(seed_text)
    print(seed_text)
    for _ in range(next_words):
        token_list = tokenizer_pos.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_freq = model.predict(token_list)
        print(predicted_freq)
        predicted = model.predict_classes(token_list, verbose=0)
        print(predicted)
        output_word = ""
        for word,index in tokenizer_pos.word_index.items():
            if index == predicted:
                print('index:', index, 'type of index:', type(index), 'predicted:', predicted,  'type of predicted:', type(predicted))
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [156]:
seed_text = "Short"

In [157]:
generate_text(seed_text, 1, model, max_sequence_len)

ADJ
[[0.00079525 0.51307446 0.31668687 0.07404561 0.04546315 0.01399473
  0.00418766 0.01076452 0.00992534 0.00647723 0.00257796 0.00200722]]
[1]
index: 1 type of index: <class 'int'> predicted: [1] type of predicted: <class 'numpy.ndarray'>


'Adj Noun'

In [135]:
max_sequence_len = max([len(x) for x in input_sequences_pos])
input_sequences_pos = np.array(pad_sequences(input_sequences_pos, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences_pos[:,:-1],input_sequences_pos[:,1:]
for i, j in zip(predictors, label):
    print(i, j)
# print(label)
label = ku.to_categorical(label, num_classes=total_words_pos)

[0 0 0 0 0 0 2] [0 0 0 0 0 2 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 3]
[0 0 0 0 0 2 3] [0 0 0 0 2 3 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 1]
[0 0 0 0 0 3 1] [0 0 0 0 3 1 1]
[0 0 0 0 0 0 1] [0 0 0 0 0 1 1]
[0 0 0 0 0 1 1] [0 0 0 0 1 1 1]
[0 0 0 0 0 0 1] [0 0 0 0 0 1 1]
[0 0 0 0 0 1 1] [0 0 0 0 1 1 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 2]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 2]
[0 0 0 0 0 2 2] [0 0 0 0 2 2 1]
[0 0 0 0 0 0 1] [0 0 0 0 0 1 1]
[0 0 0 0 0 1 1] [0 0 0 0 1 1 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 2]
[0 0 0 0 0 3 2] [0 0 0 0 3 2 2]
[0 0 0 0 3 2 2] [0 0 0 3 2 2 2]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 2]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 1]
[0 0 0 0 0 2 1] [0 0 0 0 2 1 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 1]
[0 0 0 0 0 2 1] [0 0 0 0 2 1 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 3]
[0 0 0 0 0 3 3] [0 0 0 0 3 3 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 1]
[0 0 0 0 0 3 1] [0 0 0 0 3 1 1]
[0 0 0 0 3 1 1] [0 0 0 3 1 1 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 1]
[0 0 0 0 0 3 1] [0 0 0 0 3 1 1]
[0 0 0 0

[0 0 0 0 0 0 3] [0 0 0 0 0 3 2]
[0 0 0 0 0 3 2] [0 0 0 0 3 2 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 2]
[0 0 0 0 0 2 2] [0 0 0 0 2 2 2]
[0 0 0 0 2 2 2] [0 0 0 2 2 2 2]
[0 0 0 2 2 2 2] [0 0 2 2 2 2 2]
[0 0 2 2 2 2 2] [0 2 2 2 2 2 1]
[0 2 2 2 2 2 1] [2 2 2 2 2 1 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 1]
[0 0 0 0 0 3 1] [0 0 0 0 3 1 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 2]
[0 0 0 0 0 2 2] [0 0 0 0 2 2 2]
[0 0 0 0 2 2 2] [0 0 0 2 2 2 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 2]
[0 0 0 0 0 2 2] [0 0 0 0 2 2 2]
[0 0 0 0 2 2 2] [0 0 0 2 2 2 1]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 1]
[0 0 0 0 0 3 1] [0 0 0 0 3 1 1]
[0 0 0 0 0 0 4] [0 0 0 0 0 4 2]
[0 0 0 0 0 4 2] [0 0 0 0 4 2 2]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 1]
[0 0 0 0 0 0 3] [0 0 0 0 0 3 2]
[0 0 0 0 0 3 2] [0 0 0 0 3 2 7]
[0 0 0 0 3 2 7] [0 0 0 3 2 7 1]
[0 0 0 3 2 7 1] [0 0 3 2 7 1 3]
[0 0 3 2 7 1 3] [0 3 2 7 1 3 2]
[0 0 0 0 0 0 2] [0 0 0 0 0 2 2]
[0 0 0 0 0 2 2] [0 0 0 0 2 2 3]
[0 0 0 0 2 2 3] [0 0 0 2 2 3 1]
[0 0 0 2

In [136]:
label[0]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [137]:
label.shape

(1089, 7, 12)

In [138]:
len(label)

1089

In [139]:
def create_model_time_sequenced(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(TimeDistributed(Dense(total_words, activation='softmax')))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
    
    return model

In [140]:
model_time_sequenced = create_model_time_sequenced(max_sequence_len, total_words_pos)
model_time_sequenced.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 7, 10)             120       
_________________________________________________________________
lstm_15 (LSTM)               (None, 7, 100)            44400     
_________________________________________________________________
dropout_14 (Dropout)         (None, 7, 100)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 12)             1212      
Total params: 45,732
Trainable params: 45,732
Non-trainable params: 0
_________________________________________________________________


In [141]:
model_time_sequenced.fit(predictors, label, epochs=5)

Epoch 1/5
35/35 [==============================] - 2s 4ms/step - loss: 2.3884 - accuracy: 0.5324
Epoch 2/5
35/35 [==============================] - 0s 4ms/step - loss: 1.7264 - accuracy: 0.6956
Epoch 3/5
35/35 [==============================] - 0s 4ms/step - loss: 1.1103 - accuracy: 0.7324
Epoch 4/5
35/35 [==============================] - 0s 4ms/step - loss: 0.9389 - accuracy: 0.7385
Epoch 5/5
35/35 [==============================] - 0s 4ms/step - loss: 0.8534 - accuracy: 0.7577


In [174]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    seed_text = convert_into_pos_tag(seed_text)
    print(seed_text)
    for _ in range(next_words):
        token_list = tokenizer_pos.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        print('tok list:', token_list)
        predicted_freq = model.predict(token_list)
        print(predicted_freq)
        prediction_probs = []
        for lev_1 in predicted_freq:
            for lev_2 in lev_1:
                print(np.max(lev_2))
        predicted = model.predict_classes(token_list, verbose=0)
        print(predicted)
        output_word = ""
        for pred in predicted:
            for ind_pred in pred:
                for word,index in tokenizer_pos.word_index.items():
                    if index == ind_pred:
                        output_word = word
                        break
        seed_text += " "+output_word
    return seed_text.title()

In [191]:
seed_text = "black long sleeves jerkin coat"

In [192]:
generate_text(seed_text, 1, model_time_sequenced, max_sequence_len)

PROPN PROPN NOUN
tok list: [[0 0 0 0 2 2 1]]
[[[6.8514436e-01 4.5483284e-02 5.8040928e-02 4.5131098e-02 2.9977221e-02
   1.7764792e-02 2.1879632e-02 1.9031722e-02 1.9743947e-02 1.8151848e-02
   2.1964258e-02 1.7686946e-02]
  [9.8009044e-01 3.0210624e-03 7.0579494e-03 4.9185180e-03 1.6779311e-03
   3.5106187e-04 7.0267264e-04 5.1020097e-04 5.1328627e-04 3.8734407e-04
   4.7395812e-04 2.9559579e-04]
  [9.7568685e-01 2.7332290e-03 1.0236261e-02 7.0809363e-03 1.9537951e-03
   1.9063181e-04 6.4676657e-04 4.3224051e-04 4.6581801e-04 2.1467042e-04
   2.3327401e-04 1.2564582e-04]
  [8.6594397e-01 1.4891519e-02 5.8177587e-02 3.8767893e-02 1.0969391e-02
   8.6136808e-04 3.1585742e-03 2.3844568e-03 2.6242537e-03 9.0577040e-04
   8.8912458e-04 4.2609699e-04]
  [9.9390522e-02 1.7265041e-01 3.6187300e-01 1.9418553e-01 7.4517138e-02
   1.2491855e-02 2.1891501e-02 2.2155134e-02 1.9639721e-02 1.0128277e-02
   7.3939045e-03 3.6830222e-03]
  [8.9655528e-03 3.6812809e-01 3.3678013e-01 1.3091272e-01 6.0594

'Propn Propn Noun Noun'

In [190]:
a = np.array([1])

In [159]:
type(a)

numpy.ndarray

In [160]:
a

array([1])

In [161]:
a == 1

array([ True])